In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import random
import gurobipy as gp
from gurobipy import GRB


In [2]:
#numAsset = 3
#df1 = pd.DataFrame([0.1,0.2,0.3])
#Q = np.mat(np.random.normal(0,1,(3,3)))
#Q = Q*np.transpose(Q)/1000+0.5*np.identity(numAsset)/1000
#print(Q)

#Q = df.cov()

In [3]:
df_exR = pd.read_csv("exR_file.csv")
df_exR
ticker = list(df_exR)[1:] #the first is the time stamp
print(ticker)

['XSP.TO', 'XIU.TO', 'XIN.TO', 'XBB.TO', 'XGB.TO', 'XRB.TO', 'XRE.TO']


In [4]:
df1= df_exR.iloc[:,1:4]
df2 = df_exR.iloc[:,4:8]
asset_1_index = [0,1,2]
asset_2_index = [3,4,5,6]
print(df1)
print(df2)


       XSP.TO    XIU.TO    XIN.TO
0         NaN       NaN       NaN
1   -0.037262 -0.035934 -0.059806
2    0.002846 -0.014420  0.007787
3   -0.002867  0.018924 -0.001692
4    0.017057  0.018122  0.024081
..        ...       ...       ...
178  0.027248  0.036481  0.027873
179  0.044082  0.052970  0.050659
180  0.050627  0.023047  0.011021
181  0.004213  0.030843  0.022821
182  0.021196  0.033390  0.001299

[183 rows x 3 columns]
       XBB.TO    XGB.TO    XRB.TO    XRE.TO
0         NaN       NaN       NaN       NaN
1   -0.000070       NaN -0.013703 -0.002474
2   -0.020828       NaN -0.021193 -0.009137
3    0.028914       NaN  0.040237  0.054973
4    0.014012       NaN  0.015380  0.025324
..        ...       ...       ...       ...
178 -0.027362 -0.031432 -0.042828  0.038720
179 -0.012869 -0.014480 -0.008564  0.047724
180  0.000274  0.001319 -0.020003  0.046757
181  0.005995  0.005625  0.034407  0.026312
182  0.010679  0.011981  0.009638  0.033735

[183 rows x 4 columns]


In [5]:

total_df = pd.concat([df1,df2], axis=1)# merge the 4 asset class
print(total_df)
print(total_df.shape)

print(total_df.shape[1]) # get the amount of columns of the dataframe
numAsset = total_df.shape[1]
print("number of asset is ", numAsset)
Q = total_df.cov()
Q.shape

       XSP.TO    XIU.TO    XIN.TO    XBB.TO    XGB.TO    XRB.TO    XRE.TO
0         NaN       NaN       NaN       NaN       NaN       NaN       NaN
1   -0.037262 -0.035934 -0.059806 -0.000070       NaN -0.013703 -0.002474
2    0.002846 -0.014420  0.007787 -0.020828       NaN -0.021193 -0.009137
3   -0.002867  0.018924 -0.001692  0.028914       NaN  0.040237  0.054973
4    0.017057  0.018122  0.024081  0.014012       NaN  0.015380  0.025324
..        ...       ...       ...       ...       ...       ...       ...
178  0.027248  0.036481  0.027873 -0.027362 -0.031432 -0.042828  0.038720
179  0.044082  0.052970  0.050659 -0.012869 -0.014480 -0.008564  0.047724
180  0.050627  0.023047  0.011021  0.000274  0.001319 -0.020003  0.046757
181  0.004213  0.030843  0.022821  0.005995  0.005625  0.034407  0.026312
182  0.021196  0.033390  0.001299  0.010679  0.011981  0.009638  0.033735

[183 rows x 7 columns]
(183, 7)
7
number of asset is  7


(7, 7)

In [6]:
def formDf(x,ticker):
    x = pd.DataFrame(x).T
    x.columns = ticker
    return x

In [7]:
#Equal Weight strategy
weight_eq = pd.DataFrame(np.repeat(1/numAsset, numAsset))
print(weight_eq)

x_ew = formDf(weight_eq,ticker) # result for equal weight
print(x_ew)

          0
0  0.142857
1  0.142857
2  0.142857
3  0.142857
4  0.142857
5  0.142857
6  0.142857
     XSP.TO    XIU.TO    XIN.TO    XBB.TO    XGB.TO    XRB.TO    XRE.TO
0  0.142857  0.142857  0.142857  0.142857  0.142857  0.142857  0.142857


In [ ]:
# Risk Parity


#define initial portfolio (equal weighed)
#kappa = 5 #initial arbitrary constant
#y0 = np.transpose(np.mat((np.repeat(1/numAsset,numAsset))))


In [ ]:
#create a new model
#model_1 = gp.Model("matrix1")

#create variables
#y = model_1.addMVar(shape = 7, vtype=GRB.CONTINUOUS,lb = 0,name ="y" )

#set objective
#obj = 

In [8]:
# Mean Variance optimization
def meanVarianceOpt(numAsset):
    def objective(weight):#weight
        weight = np.array(weight)
        return weight.dot(Q).dot(weight.T)

    #constraints
    cons = ({'type':"eq",'fun':lambda x: np.sum(x)-1},
           )
    
    #long only
    bounds = tuple((0,1) for x in range(numAsset))
    
    #initialize the weights with an even split, equal weights
    guess = pd.DataFrame(np.repeat(1/numAsset, numAsset))
    optimized_result = minimize(objective, guess, method = "SLSQP", bounds  = bounds, constraints= cons)
    
    return optimized_result

In [9]:
x_mvo = meanVarianceOpt(numAsset).x
x_mvo = formDf(x_mvo,ticker)

In [10]:
x_mvo

,XSP.TO,XIU.TO,XIN.TO,XBB.TO,XGB.TO,XRB.TO,XRE.TO
0,2.081668e-17,0.052737,0.017391,0.327339,0.346904,0.243243,0.012386


In [11]:
x_ew

,XSP.TO,XIU.TO,XIN.TO,XBB.TO,XGB.TO,XRB.TO,XRE.TO
0,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857,0.142857
